In [52]:
import pandas as pd
import numpy as np
import json

In [53]:
categories = ['Children', 'Biography', 'Comics', 'Fantasy Paranormal', 'Mystery Thriller Crime', 'Poetry', 'Young Adult', 'Romance']
# categories = ['Children', 'Comics', 'Mystery Thriller Crime', 'Poetry', 'Young Adult', 'Romance']

In [54]:
total_df = pd.DataFrame(columns = ['user_id', 'book_id', 'rating', 'review_text', 'n_votes'] )
# categories = ['Children']
for category in categories:
    interactions_df = pd.read_csv(f"/Volumes/Drive/allenlu/Google Drive/My Drive/258/sampled/Goodreads_{category}_Interactions_sampled.csv")
    reviews_df = pd.read_csv(f"/Volumes/Drive/allenlu/Google Drive/My Drive/258/sampled/Goodreads_{category}_Reviews_sampled.csv")
    
    reviews_df['n_votes'] = reviews_df['n_votes'] + 1
    merged_df = pd.merge(
        interactions_df,
        reviews_df[['user_id', 'book_id', 'rating', 'review_text', 'n_votes']],
        on=['user_id', 'book_id', 'rating'],
        how='left'
    )
    merged_df['n_votes'] = merged_df['n_votes'].fillna(0)
    merged_df['review_text'] = merged_df['review_text'].fillna('')
    merged_df.drop('is_read', axis=1, inplace=True)
    print(f"{category}, {len(interactions_df['book_id'].unique())}, {len(reviews_df['book_id'].unique())}, {len(merged_df['book_id'].unique())}")
    total_df = pd.concat([total_df, merged_df])


Children, 4995, 4997, 4995


/var/folders/cv/r6drvdzs40v0w7d6c9p4mbp80000gn/T/ipykernel_58580/500725024.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_df = pd.concat([total_df, merged_df])


Biography, 4997, 4998, 4997
Comics, 4999, 4999, 4999
Fantasy Paranormal, 4993, 4997, 4993
Mystery Thriller Crime, 4996, 5000, 4996
Poetry, 4995, 4994, 4995
Young Adult, 4992, 5000, 4992
Romance, 4996, 4999, 4996


In [55]:
def filter_active_users_and_books(df, min_user_reviews=10, min_book_reviews=5):

    user_review_counts = df['user_id'].value_counts()
    active_users = user_review_counts[user_review_counts >= min_user_reviews].index
    user_filtered_df = df[df['user_id'].isin(active_users)]
    
    book_review_counts = user_filtered_df['book_id'].value_counts()
    active_books = book_review_counts[book_review_counts >= min_book_reviews].index
    final_filtered_df = user_filtered_df[user_filtered_df['book_id'].isin(active_books)]
    
    final_filtered_df = final_filtered_df.reset_index(drop=True)
    
    print("User Statistics:")
    print(f"Original number of users: {df['user_id'].nunique()}")
    print(f"Number of users after filtering: {final_filtered_df['user_id'].nunique()}")
    print(f"Min user reviews: {final_filtered_df['user_id'].value_counts().min()}")
    print(f"Max user reviews: {final_filtered_df['user_id'].value_counts().max()}")
    
    print("\nBook Statistics:")
    print(f"Original number of books: {df['book_id'].nunique()}")
    print(f"Number of books after filtering: {final_filtered_df['book_id'].nunique()}")
    print(f"Min book reviews: {final_filtered_df['book_id'].value_counts().min()}")
    print(f"Max book reviews: {final_filtered_df['book_id'].value_counts().max()}")
    
    print(f"\nTotal number of reviews after filtering: {len(final_filtered_df)}")
    
    return final_filtered_df

def calculate_user_book_statistics(df):

    user_stats = {}
    book_stats = {}
    
    user_groups = df.groupby('user_id')['rating']
    user_means = user_groups.mean()
    user_vars = user_groups.var()

    for user_id in user_means.index:
        user_stats[user_id] = {
            'mean': user_means[user_id],
            'variance': user_vars[user_id] if not pd.isna(user_vars[user_id]) else 0
        }
    
    book_groups = df.groupby('book_id')['rating']
    book_means = book_groups.mean()
    book_vars = book_groups.var()

    for book_id in book_means.index:
        book_stats[book_id] = {
            'mean': book_means[book_id],
            'variance': book_vars[book_id] if not pd.isna(book_vars[book_id]) else 0
        }
    
    return user_stats, book_stats

def save_stats(user_stats, book_stats):
    user_stats_clean = {
        str(user_id): {
            'mean': float(stats['mean']),
            'variance': float(stats['variance'])
        }
        for user_id, stats in user_stats.items()
    }
    
    book_stats_clean = {
        str(book_id): {
            'mean': float(stats['mean']),
            'variance': float(stats['variance'])
        }
        for book_id, stats in book_stats.items()
    }
    
    # Save user statistics
    with open('user_stats.json', 'w') as f:
        json.dump(user_stats_clean, f, indent=4)
    
    # Save book statistics
    with open('book_stats.json', 'w') as f:
        json.dump(book_stats_clean, f, indent=4)

def count_books_by_genre(df):
    # Group by genre and count unique book_ids
    genre_counts = df.groupby('genre')['book_id'].nunique()
    
    # Sort values in descending order (optional)
    genre_counts = genre_counts.sort_values(ascending=False)
    
    # Print results
    print("Number of unique books in each genre:")
    for genre, count in genre_counts.items():
        print(f"{genre}: {count} books")
        
    return genre_counts

In [56]:
#filter the dataset further since we don't want books that only have 1 rating or users who only rated one book
filter_df = filter_active_users_and_books(total_df, 10, 5)
print('-----------------------------------------')

User Statistics:
Original number of users: 615068
Number of users after filtering: 195667
Min user reviews: 5
Max user reviews: 1215

Book Statistics:
Original number of books: 39514
Number of books after filtering: 31220
Min book reviews: 5
Max book reviews: 95167

Total number of reviews after filtering: 4993063
-----------------------------------------


In [57]:
user_statistics, book_statistics = calculate_user_book_statistics(filter_df)
save_stats(user_statistics, book_statistics)
filter_df.to_csv('dataset.csv', index = False)

In [58]:
#merge subset data
df_books = pd.DataFrame(columns= ['book_id', 'title', 'average_rating', 'rating_variance', 'publication_year', 'num_pages', 'ratings_count', 'text_reviews_count','Children', 'Biography', 'Comics', 'Fantasy Paranormal', 'Mystery Thriller Crime', 'Poetry', 'Young Adult', 'Romance'])
for category in categories:
    file_path = f"/Volumes/Drive/allenlu/Google Drive/My Drive/258/sampled/Goodreads {category} subset.csv"
    columns = ['book_id', 'title', 'average_rating', 'publication_year', 'num_pages', 'ratings_count', 'text_reviews_count']
    df = pd.read_csv(file_path)
    df = df[columns]
    df[category] = 1
    df['rating_variance'] = 0
    df_books = pd.concat([df_books, df])

    

/var/folders/cv/r6drvdzs40v0w7d6c9p4mbp80000gn/T/ipykernel_58580/3857239204.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_books = pd.concat([df_books, df])


In [59]:
review_book_ids = filter_df['book_id'].unique()

filtered_books = df_books[df_books['book_id'].isin(review_book_ids)]

In [60]:
merged_books = filtered_books.groupby('book_id').agg({
    'title': 'first',  # Take the first title since it should be the same
    'publication_year': 'first',  # Take the first year
    'num_pages': 'first',  # Take the first number of pages
    'average_rating': 'first',
    'rating_variance': 'first',
    'ratings_count': 'first',  # Take the first ratings count
    'text_reviews_count': 'first',  # Take the first text reviews count
    'Children': 'max',  # Take max for genre columns (0 or 1)
    'Biography': 'max',
    'Comics': 'max',
    'Fantasy Paranormal': 'max',
    'Mystery Thriller Crime': 'max',
    'Poetry': 'max',
    'Young Adult': 'max',
    'Romance': 'max'
}).reset_index()

# Verify if the merging worked correctly
print("Original shape:", filtered_books.shape)
print("Merged shape:", merged_books.shape)

# To check if any book still has multiple genres (there should be)
genre_columns = ['Children', 'Biography', 'Comics', 'Fantasy Paranormal', 
                'Mystery Thriller Crime', 'Poetry', 'Young Adult', 'Romance']
genres_per_book = merged_books[genre_columns].sum(axis=1)
print("\nNumber of genres per book:")
print(genres_per_book.value_counts().sort_index())

Original shape: (31643, 16)
Merged shape: (31220, 16)

Number of genres per book:
1.0    30798
2.0      421
3.0        1
Name: count, dtype: int64


In [61]:
merged_books.fillna(0)

/var/folders/cv/r6drvdzs40v0w7d6c9p4mbp80000gn/T/ipykernel_58580/3887039245.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_books.fillna(0)


,book_id,title,publication_year,num_pages,average_rating,rating_variance,ratings_count,text_reviews_count,Children,Biography,Comics,Fantasy Paranormal,Mystery Thriller Crime,Poetry,Young Adult,Romance
0,5,Harry Potter and the Prisoner of Azkaban (Harr...,2004.0,435.0,4.53,0,1876252,28561,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,214,Sideswipe: A Hoke Moseley Novel,2005.0,215.0,4.04,0,600,57,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,244,The Puffin Book Of Nonsense Verse,1996.0,287.0,4.04,0,63,6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,289,The Beloved: Reflections on the Path of the Heart,1998.0,102.0,4.18,0,300,16,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,290,Jesus the Son of Man,1995.0,216.0,3.99,0,1174,71,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31215,36348325,Nowhere to Run,0.0,264.0,4.22,0,16,10,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
31216,36359242,Bluecollar Bear (Black Oak Bears #1),0.0,0.0,4.23,0,149,44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
31217,36359314,"How to Heal a Life (The Haven, #2)",2017.0,0.0,4.05,0,45,16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
31218,36396964,Everything We Left Behind,2017.0,350.0,4.04,0,730,37,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [62]:
for idx, row in merged_books.iterrows():
    merged_books.at[idx,'rating_variance'] =  book_statistics[row['book_id']]['variance']

In [63]:
merged_books.to_csv('books_data.csv', index = False)